# Perplexity 모델 No-tuned

In [ ]:
# 'input_text': '원고가 소속회사의 노동조합에서 분규가 발생하자 노조활동을 구실로 정상적인 근무를 해태하고, 노조조합장이 사임한 경우, 노동조합규약에 동 조합장의 직무를 대행할 자를 규정해 두고 있음에도 원고 자신이 주동하여 노조자치수습대책위원회를 구성하여 그 위원장으로 피선되어 근무시간중에도 노조활동을 벌여 운수업체인 소속회사의 업무에 지장을 초래하고 종업원들에게도 나쁜 영향을 끼쳐 소속회사가 취업규칙을 위반하고 고의로 회사업무능률을 저해하였으며 회사업무상의 지휘명령에 위반하였음을 이유로 원고를 징계해고 하였다면, 이는 원고의 노동조합 활동과는 관계없이 회사취업규칙에 의하여 사내질서를 유지하기 위한 사용자 고유의 징계권에 기하여 이루어진 정당한 징계권의 행사로 보아야 한다.'
# 'target_text': '원고가  주동하여 회사업무능률을 저해하고 회사업무상의 지휘명령에 위반하였다면 이에 따른 징계해고는 사내질서를 유지하기 위한 사용자 고유의 정당한 징계권의 행사로 보아야 한다.'}

## 1. 모델 및 토크나이저 로드

In [3]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import main  # main.py에서 access_token 가져오기

# GPU 설정 및 최적화
DEVICE = "cuda:1" if torch.cuda.is_available() else "cpu"
torch.set_float32_matmul_precision('medium')
torch.backends.cudnn.benchmark = True

# Hugging Face 모델명 설정
MODEL_NAME = "perplexity-ai/r1-1776"

# Access Token 가져오기 (main.py에서 import)
ACCESS_TOKEN = main.access_token


ImportError: /home/wanted-1/miniconda3/envs/dl_project/lib/python3.9/site-packages/torch/lib/../../nvidia/cusparse/lib/libcusparse.so.12: undefined symbol: __nvJitLinkComplete_12_4, version libnvJitLink.so.12

## 2. 모델 및 토크나이저 로드

In [ ]:
def load_model():
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            MODEL_NAME, 
            use_auth_token=ACCESS_TOKEN, 
            local_files_only=False,
            trust_remote_code=True  # 사용자 정의 코드 실행 허용
        )
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME, 
            use_auth_token=ACCESS_TOKEN,
            torch_dtype=torch.float16,
            trust_remote_code=True  # 사용자 정의 코드 실행 허용
        ).to(DEVICE)
        return model, tokenizer
    except Exception as e:
        print(f"🚨 모델 로드 중 오류 발생: {e}")
        exit()


## 3. 요약 생성 함수

In [ ]:
def generate_summary(input_text, model, tokenizer, max_new_tokens=100):
    inputs = tokenizer(input_text, return_tensors="pt").to(DEVICE)

    with torch.no_grad(), torch.cuda.amp.autocast():  # 혼합 정밀도 사용
        outputs = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)


## 4. 메인 실행 함수

In [ ]:
def main():
    model, tokenizer = load_model()

    # 요약할 입력 문장 (예시)
    input_text = "한국 경제는 글로벌 경기 둔화와 내수 부진의 영향을 받고 있습니다."

    # 요약 실행
    summary = generate_summary(input_text, model, tokenizer)

    print("\n📝 입력 텍스트:")
    print(input_text)
    print("\n📌 생성된 요약:")
    print(summary)

if __name__ == "__main__":
    main()


🚨 모델 로드 중 오류 발생: name 'AutoTokenizer' is not defined


TypeError: cannot unpack non-iterable NoneType object

: 

## 4. 성능 평가

### 1. 평가 데이터셋 로드

In [ ]:
import nltk
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.meteor_score import meteor_score
from rouge_score import rouge_scorer
from bert_score import score as bertscore

nltk.download('wordnet')
nltk.download('omw-1.4')

def evaluate_summary(predictions, references):
    """
    predictions: 모델이 생성한 요약 리스트
    references:  참조 요약 리스트 (실제 요약)
    각 항목별로 짝을 맞춰 평가
    """
    # ROUGE
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1, rouge2, rougeL = 0, 0, 0
    for pred, ref in zip(predictions, references):
        scores = scorer.score(ref, pred)
        rouge1 += scores['rouge1'].fmeasure
        rouge2 += scores['rouge2'].fmeasure
        rougeL += scores['rougeL'].fmeasure
    n = len(predictions)
    rouge_scores = {
        "rouge1": rouge1 / n,
        "rouge2": rouge2 / n,
        "rougeL": rougeL / n
    }
    
    # BLEU
    refs_list = [[ref.split()] for ref in references]
    preds_list = [pred.split() for pred in predictions]
    bleu = corpus_bleu(refs_list, preds_list)
    
    # METEOR
    total_meteor = 0
    for pred, ref in zip(predictions, references):
        total_meteor += meteor_score([ref], pred)
    meteor_avg = total_meteor / n
    
    # BERTScore (한국어)
    P, R, F1 = bertscore(predictions, references, lang="ko")
    bert_f1 = F1.mean().item()
    
    results = {
        "ROUGE": rouge_scores,
        "BLEU": bleu,
        "METEOR": meteor_avg,
        "BERTScore_F1": bert_f1
    }
    return results

# 사용 예시:
# predictions = ["모델이 생성한 요약1", "모델이 생성한 요약2", ...]
# references = ["실제 요약1", "실제 요약2", ...]
# eval_results = evaluate_summary(predictions, references)
# print(eval_results)


[nltk_data] Downloading package wordnet to /home/wanted-1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/wanted-1/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
